In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

## Data Loading
### focus on journals related to finance, economics, management, etc., specifically:
<ol>
  <li>from CABS file use all journals in spreadsheet “CABS 2021”</li>
  <li>from JCR files use all journals in spreadsheets: BUSINESS, BUSINESS, FINANCE, ECONOMICS, MANAGEMENT
</li>
  <li>from “Wykaz…” files use the journals from the following:
ekonomia i finanse		nauki o zarządzaniu i jakości
501		506;
Use only spreadsheet “czasopisma naukowe”
</li>
</ol>

Note that if the journal appears at least in one of the source, it has to be taken into account


In [14]:
# Specify the directory containing your Excel files
directory_path = 'source files'

# Get a list of all Excel files in the directory
excel_files = [file for file in os.listdir(directory_path) if file.endswith('.xlsx')]

# Create an empty dictionary to store the loaded data frames
data_frames = {}

# Load from CABS file
cabs_file = [file for file in excel_files if "CABS" in file][0]
cabs_sheet = "CABS 2021"
cabs_file_path = os.path.join(directory_path, cabs_file)
try:
    cabs_df = pd.read_excel(cabs_file_path, sheet_name=cabs_sheet)
    data_frames[cabs_file] = cabs_df
except Exception as e:
    print(f"Error loading {cabs_file}: {e}")

# Load from JCR files
jcr_files = [file for file in excel_files if "JCR" in file]
jcr_sheets = ["BUSINESS", "BUSINESS, FINANCE", "ECONOMICS", "MANAGEMENT"]
for jcr_file in jcr_files:
    jcr_file_path = os.path.join(directory_path, jcr_file)
    try:
        jcr_df = pd.read_excel(jcr_file_path, sheet_name=jcr_sheets)
        data_frames[jcr_file] = jcr_df
    except Exception as e:
        print(f"Error loading {jcr_file}: {e}")

# Load from files containing the word “Wykaz”
wykaz_files = [file for file in excel_files if "Wykaz" in file]
for wykaz_file in wykaz_files:
    wykaz_file_path = os.path.join(directory_path, wykaz_file)
    try:
        wykaz_df = None
        with pd.ExcelFile(wykaz_file_path) as xls:
            for sheet_name in xls.sheet_names:
                if "czasopisma" in sheet_name.lower():
                    wykaz_df = pd.read_excel(wykaz_file_path, sheet_name=sheet_name, engine='openpyxl')
                    data_frames[wykaz_file] = wykaz_df
                    break
        if wykaz_df is None:
            print(f"No sheet containing 'czasopisma' found in {wykaz_file}")
    except Exception as e:
        print(f"Error loading {wykaz_file}: {e}")

[autoreload of openpyxl.descriptors failed: Traceback (most recent call last):
  File "C:\Users\Alon\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Alon\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\Alon\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 393, in update_generic
    update(a, b)
  File "C:\Users\Alon\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 361, in update_class
    update_instances(old, new)
  File "C:\Users\Alon\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 319, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeError: can't apply this __setattr__ to MetaStrict object
]
[autoreload of openpyxl.styles.ali

In [19]:
data_frames.keys()

dict_keys(['CABS Journal Ranking 2021.xlsx', 'JCR_2021_ALL.xlsx', 'JCR_2022_ALL.xlsx', '20211201_Wykaz_dyscyplin_przypisanych_do_czasopism_naukowych_i_materiałów_konferencyjnych - Dec 2021 1.xlsx', '20240105_Wykaz_czasopism_naukowych_2024_styczeń.xlsx', 'Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych Feb 2021.xlsx'])